# Henry PI 2: Machine Learning

• Caros incluyen el promedio

• stacking & walking

• revisar descripciones repetidas: anuncios publicados múltiples veces

• robustscaler lidia mejor con outliers que standardscaler

• el registro más al sur parece estar en Nariño, pero también hay registros en el amazonas

## ------------- D A T A --- E X P L O R A T I O N --------------

We start by importing the libraries that we need

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from sklearn import preprocessing
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from helpers import *

In [2]:
# Next we import the dataset with the training data into a Pandas DataFrame

original_df = pd.read_csv('datasets/properties_colombia_train.csv')
#original_df.sample(5)

In [3]:
# Now we obtain some basic information about the DataFrame, along with the mean value from the feature we will use to create the target column

original_price_mean = original_df.price.mean()

print(f'• Original shape: {original_df.shape}\n')
print(f'• Original columns: {original_df.columns}\n')
print(f"• Original price column's mean: {original_price_mean}")

• Original shape: (197549, 27)

• Original columns: Index(['Unnamed: 0', 'id', 'ad_type', 'start_date', 'end_date', 'created_on',
       'lat', 'lon', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms',
       'bathrooms', 'surface_total', 'surface_covered', 'price', 'currency',
       'price_period', 'title', 'description', 'property_type',
       'operation_type', 'geometry'],
      dtype='object')

• Original price column's mean: 643605091.0064613


In [4]:
# We look for duplicated registers (spoiler: there are none)

original_df.duplicated().value_counts()

False    197549
dtype: int64

In [5]:
# We look for missing values per feature (we find a lot of them, particularly in l4, l5, l6, rooms, bedrooms, surface_total, surface_covered and price_period)

original_df.isnull().sum()

Unnamed: 0              0
id                      0
ad_type                 0
start_date              0
end_date                0
created_on              0
lat                 49498
lon                 49498
l1                      0
l2                      0
l3                  11032
l4                 152182
l5                 170140
l6                 190682
rooms              170012
bedrooms           157024
bathrooms           41082
surface_total      190575
surface_covered    187747
price                  63
currency               67
price_period       161578
title                   1
description           121
property_type           0
operation_type          0
geometry                0
dtype: int64

## ---------- G E N E R A T I N G --- T A R G E T --- C O L U M N ----------

In [6]:
# We start by creating a copy of the original dataset and looking for missing values in the 'price' column (which we can see above as well)
# Our targets will be obtained from the information contained in this column, so any training data without an associated target value will be pretty much useless.

df_Xy = original_df.copy()
df_Xy.price.isnull().sum()

63

We can see that we have 63 missing values in the 'price' column, the one we will be using to create our target classification based on it's mean value.

We procceed to drop those registers, this is because we need them to have a target value in order to train our models.

In [7]:
df_Xy.dropna(subset=['price'], inplace=True)

price_mean_after_dropna = df_Xy.price.mean()

print(f'Original DataFrame Shape: {original_df.shape}')
print(f'• DataFrame Shape (after dropna-price): {df_Xy.shape}\n')
print(f'• DataFrame Price column mean (after dropna-price): {price_mean_after_dropna}\n')
print(f"• Is the price's mean still the same as the original: {price_mean_after_dropna==original_price_mean}")

Original DataFrame Shape: (197549, 27)
• DataFrame Shape (after dropna-price): (197486, 27)

• DataFrame Price column mean (after dropna-price): 643605091.0064613

• Is the price's mean still the same as the original: True


In [8]:
# We check again for missing values

df_Xy.price.isnull().sum()

0

In [9]:
# We check for the extreme values in the column

df_Xy.price.min(), df_Xy.price.max()

(0.0, 345000000000.0)

In [10]:
# We check for the amount of appereances of these extreme values

df_Xy.price.value_counts()[0], df_Xy.price.value_counts()[345000000000.0]

(4, 1)

In [11]:
# As we found an absurdly big value as the max value of price column we check for some information about the biggest values in this column.

#df_Xy.sort_values(by='price',ascending=False).head(100).price.mean() # Output: 54,246'115,351.52
#df_Xy.sort_values(by='price',ascending=False).head(1000).price.mean() # Output: 17,829'070,843.313

#### • IMPORTANT NOTE:

Above we can see that there are extreme outliers in the column from which we are getting our training data targets. This is an important situation that must be adressed with the client, as these outliers (specially the big ones) will distort the column's mean value, affecting the division betweeen 'expensive' and 'cheap' house we are creating in our target column.

Now we will create the 'target' column using the values from 'price', separating them into two categories based on the mean of the column.

In [12]:
df_Xy['target'] = (df_Xy['price'] >= original_price_mean).astype(int)
print(df_Xy['target'].shape)
df_Xy['target'].value_counts()

(197486,)


0    151128
1     46358
Name: target, dtype: int64

In [13]:
# Now we look for the amount of different values per feature (in order to filter out redundant and non-informative features)

for x in df_Xy:
    print(f'\n• {x}:\t{len(df_Xy[x].value_counts())}')


• Unnamed: 0:	197486

• id:	197486

• ad_type:	1

• start_date:	145

• end_date:	446

• created_on:	145

• lat:	51075

• lon:	50107

• l1:	1

• l2:	31

• l3:	293

• l4:	58

• l5:	20

• l6:	146

• rooms:	29

• bedrooms:	37

• bathrooms:	20

• surface_total:	1030

• surface_covered:	781

• price:	6096

• currency:	2

• price_period:	1

• title:	94963

• description:	111312

• property_type:	8

• operation_type:	1

• geometry:	62785

• target:	2


From the output above we can see that:
1) There are several features with only one value throughout all of the 197486 registers (ad_type, l1, price_period, operation_type). This features give us no information.
2) We can see that the columns labeled 'Unnamed: 0' and the 'id' have unique values (identifiers) for each one of the rows and thus are redundant.

We will procceed to create another dataframe from the original one ignoring these features, along with the 'price' column which was only useful for us in order to obtain our 'target' column. 

After this we will check for duplicates (once we have removed the identifiers that guaranteed every row was unique) and remove them. This will give us a somewhat clean dataset to begin preprocessing our data, i.e. applying to it the changes that we would apply to any input data given to our finished model in order to get predictions from it.

In [14]:
# We create a new DataFrame which we will use to train our model with, ignoring the unnecessary columns

df_train = df_Xy.drop(['ad_type', 'l1', 'price_period', 'operation_type', 'Unnamed: 0', 'id', 'price'], axis=1)
print(f'• Training DataFrame Shape: {df_train.shape}\n')
print(f'• Training DataFrame Columns: {df_train.columns}\n')

• Training DataFrame Shape: (197486, 21)

• Training DataFrame Columns: Index(['start_date', 'end_date', 'created_on', 'lat', 'lon', 'l2', 'l3', 'l4',
       'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms', 'surface_total',
       'surface_covered', 'currency', 'title', 'description', 'property_type',
       'geometry', 'target'],
      dtype='object')



In [15]:
df_train.duplicated().value_counts()

False    193395
True       4091
dtype: int64

We can see that after dropping the redundant and identifier columns we got a total of 4091 duplicated registers.

We procceed to eliminate them.

In [16]:
df_train.drop_duplicates(inplace=True)
df_train.duplicated().value_counts()

False    193395
dtype: int64

## -------------- D A T A --- P R E P R O C E S S I N G --- 1 --------------

### ---------------------- FINDING THE APPROPIATE TRANSFORMATIONS ----------------------

In this section we will analyze our dataset's features grouping them by the type of data portrayed in them (date, location, ).

This way, we will be able to determine the best transformations to perform on each of them in order to feed our models with the best quality data we can get.

In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193395 entries, 0 to 197548
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   start_date       193395 non-null  object 
 1   end_date         193395 non-null  object 
 2   created_on       193395 non-null  object 
 3   lat              144876 non-null  float64
 4   lon              144876 non-null  float64
 5   l2               193395 non-null  object 
 6   l3               182567 non-null  object 
 7   l4               44345 non-null   object 
 8   l5               26773 non-null   object 
 9   l6               6793 non-null    object 
 10  rooms            27453 non-null   float64
 11  bedrooms         40378 non-null   float64
 12  bathrooms        153016 non-null  float64
 13  surface_total    6942 non-null    float64
 14  surface_covered  9755 non-null    float64
 15  currency         193391 non-null  object 
 16  title            193394 non-null  obje

In [18]:
print(f'• Total registers: {len(df_train)}')
print('• Null values per feature:')
df_train.isnull().sum()

• Total registers: 193395
• Null values per feature:


start_date              0
end_date                0
created_on              0
lat                 48519
lon                 48519
l2                      0
l3                  10828
l4                 149050
l5                 166622
l6                 186602
rooms              165942
bedrooms           153017
bathrooms           40379
surface_total      186453
surface_covered    183640
currency                4
title                   1
description           121
property_type           0
geometry                0
target                  0
dtype: int64

In [19]:
def get_info(feature_list, dataset=df_train, maxmin=False, stats=False):
    for x in feature_list:
        types = set()
        for y in dataset[x]:
            types.add(type(y))
        print(f'\n----- {x} -----\n •Data types: {types}\n •Missing values:')
        print(dataset[x].isnull().value_counts(),'\n')
        if maxmin:
            print(f' •Min: {dataset[x].min()}\n •Max: {dataset[x].max()}\n')
        if stats:
            print(f' •Mean: {dataset[x].mean()}\n •Median: {dataset[x].median()}\n •Mode: {dataset[x].mode()}\n')

### 1) DATE FEATURES: start_date, end_date & created_on

In [20]:
date_features = ['start_date', 'end_date', 'created_on']

get_info(date_features, maxmin=True)


----- start_date -----
 •Data types: {<class 'str'>}
 •Missing values:
False    193395
Name: start_date, dtype: int64 

 •Min: 2020-07-26
 •Max: 2020-12-31


----- end_date -----
 •Data types: {<class 'str'>}
 •Missing values:
False    193395
Name: end_date, dtype: int64 

 •Min: 2020-07-26
 •Max: 9999-12-31


----- created_on -----
 •Data types: {<class 'str'>}
 •Missing values:
False    193395
Name: created_on, dtype: int64 

 •Min: 2020-07-26
 •Max: 2020-12-31



Here we can see that the maximum value for the 'end_date' feature has wrong data, as it is supposed to be the date when the 'for sale' announcement stopped showing.

In [21]:
df_train.end_date.value_counts()

9999-12-31    11925
2020-08-27     3994
2020-11-13     3801
2020-07-27     2621
2020-11-30     2399
              ...  
2021-08-16        2
2021-09-26        2
2021-10-03        1
2021-07-04        1
2021-06-20        1
Name: end_date, Length: 446, dtype: int64

In [22]:
df_train[['start_date', 'end_date', 'created_on']].sort_values(by=['end_date', 'start_date'], ascending=False).head(11928)

,start_date,end_date,created_on
6193,2020-12-31,9999-12-31,2020-12-31
9072,2020-12-31,9999-12-31,2020-12-31
20600,2020-12-31,9999-12-31,2020-12-31
25347,2020-12-31,9999-12-31,2020-12-31
44356,2020-12-31,9999-12-31,2020-12-31
...,...,...,...
190288,2020-07-26,9999-12-31,2020-07-26
193889,2020-07-26,9999-12-31,2020-07-26
147834,2020-12-26,2021-10-18,2020-12-26
7068,2020-12-11,2021-10-18,2020-12-11


In [23]:
11925/len(df_train)

0.0616613666330567

We have 11925 wrong values in the 'end_date' feature (0.06%). 

We will try to replace them with a the average date difference (between start_date and end_date, not including the wrong values) if we use this column.

Also, we will convert this colonms to 'datetime' data type and afterwards wi will format them into timestamp format.

### 2) LOCATION FEATURES: l2, l3, l4, l5, l6, geometry, lat & lon

In [24]:
location_features = ['l2', 'l3', 'l4', 'l5', 'l6', 'geometry', 'lat', 'lon']

get_info(location_features[:-2])
get_info(location_features[-2:], maxmin=True)


----- l2 -----
 •Data types: {<class 'str'>}
 •Missing values:
False    193395
Name: l2, dtype: int64 


----- l3 -----
 •Data types: {<class 'str'>, <class 'float'>}
 •Missing values:
False    182567
True      10828
Name: l3, dtype: int64 


----- l4 -----
 •Data types: {<class 'str'>, <class 'float'>}
 •Missing values:
True     149050
False     44345
Name: l4, dtype: int64 


----- l5 -----
 •Data types: {<class 'str'>, <class 'float'>}
 •Missing values:
True     166622
False     26773
Name: l5, dtype: int64 


----- l6 -----
 •Data types: {<class 'str'>, <class 'float'>}
 •Missing values:
True     186602
False      6793
Name: l6, dtype: int64 


----- geometry -----
 •Data types: {<class 'str'>}
 •Missing values:
False    193395
Name: geometry, dtype: int64 


----- lat -----
 •Data types: {<class 'float'>}
 •Missing values:
False    144876
True      48519
Name: lat, dtype: int64 

 •Min: -32.787342
 •Max: 34.420334


----- lon -----
 •Data types: {<class 'float'>}
 •Missing values

From the output above we can see that the 'l4', 'l5' and 'l6' have more than half the values missing, so this columns must be dropped.

#### In the 'l2' feature, corresponding to Colombia's departments (their equivalent to states or provinces) we have no values missing.

#### In the case of 'l3', there are 10828 values missing (about 5.5% of the registers). We will replace the missing values with the capital of the corresponding departments obtained from 'l2'.

In [25]:
# Here we have a dictionary containing each of the 32 colombian departments as keys followed by their corresponding capitals as values.
# This list will be stored in the helpers.py file

'''
capitals = {'Amazonas': 'Leticia', 'Antioquia': 'Medellín', 'Arauca': 'Arauca', 'Atlántico': 'Barranquilla', 'Bolívar' : 'Cartagena', 'Boyacá': 'Tunja',
            'Caldas': 'Manizales', 'Caquetá': 'Florencia', 'Casanare': 'Yopal', 'Cauca': 'Popayán', 'Cesar': 'Valledupar', 'Chocó': 'Quibdó', 'Córdoba': 'Montería',
            'Cundinamarca': 'Bogotá D.C', 'Guainía': 'Puerto Inírida', 'Guaviare': 'San José del Guaviare', 'Huila': 'Neiva', 'La Guajira': 'Riohacha', 
            'Magdalena': 'Santa Marta', 'Meta': 'Villavicencio', 'Nariño': 'Pasto', 'Norte de Santander': 'Cúcuta', 'Putumayo': 'Mocoa', 'Quindío': 'Armenia',
            'Risaralda': 'Pereira', 'San Andrés Providencia y Santa Catalina': 'San Andrés', 'Santander': 'Bucaramanga', 'Sucre': 'Sincelejo', 'Tolima': 'Ibagué',
            'Valle del Cauca': 'Cali', 'Vaupés': 'Mitú', 'Vichada': 'Puerto Carreño'}
'''
len(capitals)


32

In [26]:
# This is the amount of different cities in the 'l3' feature
len(df_train.l3.unique())

294

#### Regarding the latitude and longitude values from the dataset, we can see from the output from the function at the beginning of this section that there are 48519 missing values on each of these features.


In [27]:
# Here we check whether the missing values correspond to the same registers in the dataset:

print(f"Rows missing 'lat' values: {len(df_train[df_train['lat'].isnull()])}")
print(f"Rows missing 'lon' values: {len(df_train[df_train['lon'].isnull()])}")
#print(f"Rows missing both 'lat' and 'lon' values (1): {len(df_train[df_train['lat'].isnull()][df_train['lon'].isnull()])}")
print(f"Rows missing both 'lat' and 'lon' values (2): {len(df_train[df_train['lat'].isnull()][df_train[df_train['lat'].isnull()]['lon'].isnull()])}")


Rows missing 'lat' values: 48519
Rows missing 'lon' values: 48519
Rows missing both 'lat' and 'lon' values (2): 48519


In [28]:
# Here we take a sample to further proof that every row missing a 'lat' value is missing it's 'lon' value as well
df_train[df_train['lat'].isnull()].sample(10)

,start_date,end_date,created_on,lat,lon,l2,l3,l4,l5,l6,...,bedrooms,bathrooms,surface_total,surface_covered,currency,title,description,property_type,geometry,target
100086,2020-12-23,2020-12-25,2020-12-23,NaN,NaN,Antioquia,Medellín,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,COP,Apartamento en Venta Ubicado en MEDELLIN,Codigo Inmueble 273 Se vende apartamento en Ar...,Apartamento,POINT EMPTY,0
168389,2020-12-30,2020-12-31,2020-12-30,NaN,NaN,Antioquia,Medellín,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,COP,Terreno en Venta Ubicado en CARMEN DE VIBORAL,Codigo Inmueble 6528 Hermoso lote con una exce...,Lote,POINT EMPTY,1
69395,2020-10-09,2020-10-10,2020-10-09,NaN,NaN,Antioquia,Medellín,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,COP,Terreno en Venta Ubicado en ENVIGADO,Codigo Inmueble 6394 Excelente oportunidad de ...,Lote,POINT EMPTY,1
7396,2020-10-26,2020-10-27,2020-10-26,NaN,NaN,Antioquia,Medellín,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,COP,Apartamento en Venta Ubicado en RIONEGRO,Codigo Inmueble 6323 Cómodo apartamento con 3 ...,Apartamento,POINT EMPTY,0
13445,2020-11-06,2020-11-07,2020-11-06,NaN,NaN,Antioquia,Medellín,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,COP,Apartamento en Venta Ubicado en SABANETA,Codigo Inmueble 4018 APARTAMENTO CON 3 ALCOBAS...,Apartamento,POINT EMPTY,0
113034,2020-07-27,2020-07-27,2020-07-27,NaN,NaN,Caldas,Manizales,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,COP,CASA EN VENTA EN SAN BERNARDO DEL VIENTO MANI...,Casa con un área de 3200 mts cuenta con 5 alco...,Casa,POINT EMPTY,1
82541,2020-11-29,2020-12-03,2020-11-29,NaN,NaN,Antioquia,Medellín,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,COP,Casa en Venta Ubicado en SABANETA,Codigo Inmueble 169 Casa de segundo piso ubica...,Casa,POINT EMPTY,0
193075,2020-10-07,2020-10-08,2020-10-07,NaN,NaN,Antioquia,Medellín,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,COP,Apartamento en Venta Ubicado en MEDELLIN,Codigo Inmueble 972 APARTAMENTO MODERNO de 51 ...,Apartamento,POINT EMPTY,0
56046,2020-12-03,2021-03-01,2020-12-03,NaN,NaN,Valle del Cauca,Cali,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,COP,Apartamento En Venta En Cali Santa Teresita Co...,VENTA ESPECTACULAR APTO EN SANTA TERESITA| PIS...,Apartamento,POINT EMPTY,1
107821,2020-11-23,2021-02-22,2020-11-23,NaN,NaN,Atlántico,Barranquilla,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,COP,APARTAMENTO EN ZONA CIUDAD JARDIN | PARA ESTRENAR,"Apartamento en venta, con área de 121 m2, sala...",Apartamento,POINT EMPTY,0


#### In order to analize the latitudes and longitudes from the rows with a value for these columns, we need to define certain limits for the colombian territory, beyond which we shouldn't expect to find any lat or lon values.

![Colombia Latitudes and Longitudes](https://i.imgur.com/ZdKWfRG.png)

In [29]:
# We define the corresponding limits as two lists, one for the latitudes and one for the longitudes
# This limits encompass the colombian insular territories, which extend further west an north than it's continental territory

lat_col = [-4.5, 15]    # Southernmost and northernmost latitudes respectively
lon_col = [-82, -67]    # Westernmost and easternmost longitudes respectively

In [30]:
count_lat_smaller = 0   # Registers with a latitude to the south of Colombia
count_lat_greater = 0   # Registers with a latitude to the north of Colombia
for x in df_train.lat:
    if x<lat_col[0]:
        count_lat_smaller += 1
    elif x>lat_col[1]:
        count_lat_greater += 1
    
print(f'• Latitudes south from Colombia: {count_lat_smaller}\n• Latitudes north from Colombia: {count_lat_greater}')

• Latitudes south from Colombia: 1
• Latitudes north from Colombia: 1


As we can see, there's only 1 value exceeding Colombia's latitudes on each direction in our dataset. We can visualize them:

In [31]:
# 
df_train.sort_values(by='lat').head(2)
#df_Xy.sort_values(by='price',ascending=False).head(100).price.mean() # Output: 54,246'115,351.52

,start_date,end_date,created_on,lat,lon,l2,l3,l4,l5,l6,...,bedrooms,bathrooms,surface_total,surface_covered,currency,title,description,property_type,geometry,target
138682,2020-09-29,2021-07-26,2020-09-29,-32.787342,-71.20732,Cundinamarca,La Calera,NaN,NaN,NaN,...,NaN,6.0,NaN,NaN,COP,51548 LA CALERA MIRADO DEL LAGO,"Casa hermosa,amplia, vigilancia sector&nbsp; t...",Casa,POINT (-71.20732 -32.787342),1
177722,2020-11-19,9999-12-31,2020-11-19,0.823972,-77.62271,Nariño,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,COP,Se vende casa en Ipiales,Venta de casa en colina verde\nInfo: 320302104...,Casa,POINT (-77.6227098 0.823972),0


In [32]:
df_train.sort_values(by='lat', ascending=False).head(2)

,start_date,end_date,created_on,lat,lon,l2,l3,l4,l5,l6,...,bedrooms,bathrooms,surface_total,surface_covered,currency,title,description,property_type,geometry,target
148562,2020-11-06,2021-07-26,2020-11-06,34.420334,-119.69819,Cundinamarca,Bogotá D.C,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,COP,51599 SANTA BARBARA APARTAMENTO 506,"Apartamento&nbsp; para&nbsp; remodelar , duple...",Apartamento,POINT (-119.69819 34.420334),0
7159,2020-09-02,2020-09-02,2020-09-02,13.351917,-81.35745,San Andrés Providencia y Santa Catalina,Providencia,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,COP,Lote Terreno en Venta en Providencia _ wasi150...,De la Isla de San Andrés en Avión son 20 minut...,Lote,POINT (-81.35745049 13.35191746),1


In [33]:
count_lon_smaller = 0   # Registers with a longitude to the west of Colombia
count_lon_greater = 0   # Registers with a longitude to the east of Colombia
for x in df_train.lon:
    if x<lon_col[0]:
        count_lon_smaller += 1
    elif x>lon_col[1]:
        count_lon_greater += 1
    
print(f'• Longitudes to the west from Colombia: {count_lon_smaller}\n• Longitudes to the east from Colombia: {count_lon_greater}')

• Longitudes to the west from Colombia: 1
• Longitudes to the east from Colombia: 0


We found only one missplaced longitude, to the west of Colombia. Now we visualize it:

In [34]:
df_train.sort_values(by='lon').head(2)

,start_date,end_date,created_on,lat,lon,l2,l3,l4,l5,l6,...,bedrooms,bathrooms,surface_total,surface_covered,currency,title,description,property_type,geometry,target
148562,2020-11-06,2021-07-26,2020-11-06,34.420334,-119.69819,Cundinamarca,Bogotá D.C,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,COP,51599 SANTA BARBARA APARTAMENTO 506,"Apartamento&nbsp; para&nbsp; remodelar , duple...",Apartamento,POINT (-119.69819 34.420334),0
114121,2020-09-16,9999-12-31,2020-09-16,12.524494,-81.72839,San Andrés Providencia y Santa Catalina,NaN,NaN,NaN,NaN,...,8.0,NaN,NaN,NaN,COP,Casa de Lujo en la Isla de Andrés,"Para vivir o para invertir, ésta casa llena de...",Finca,POINT (-81.7283900951 12.5244938806),1


In [35]:
df_train.sort_values(by='lon', ascending=False).head(2)

,start_date,end_date,created_on,lat,lon,l2,l3,l4,l5,l6,...,bedrooms,bathrooms,surface_total,surface_covered,currency,title,description,property_type,geometry,target
59361,2020-08-01,2020-09-16,2020-08-01,6.189912,-67.482570,Vichada,Puerto Carreño,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,COP,FINCA AGROINDUSTRIAL PUERTO CARREÑO,FINCA AGROINDUSTRIAL Y GANADERA A 10 MINUTOS...,Otro,POINT (-67.4825696 6.1899117),1
2232,2020-10-03,2020-11-30,2020-10-03,3.870204,-67.924336,Guainía,Inírida,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,COP,SE VENDE FINCA EN INIRIDA GUAINIA,"Finca de 14,56 hectáreas en venta en la zona r...",Lote,POINT (-67.9243361 3.8702044),0


#### There where only 3 misplaced latitude and longitude values in total. Those can be replaced by the coordinates from the city in the register ('l3' value).

In [36]:
# Here we get a pd.Series with the possible combinatorics of the values in 'l2' and 'l3' for each row.

combinations = []
for x in range(len(df_train)):
    if str(df_train.iloc[x].l3) != 'nan':
        combinations.append(f'{df_train.iloc[x].l3}, {df_train.iloc[x].l2}')
comb_series = pd.Series(combinations)
unique_l2_l3 = comb_series.unique()

In [37]:
print(len(unique_l2_l3))

299


In [38]:
df_cities = df_train.l3.unique()
print(f'Amount of different cities in df_train.l3: {len(df_cities)}')
print(f'Amount of different cities in the combination df_train.l3-df_train.l2: {len(unique_l2_l3)}')

Amount of different cities in df_train.l3: 294
Amount of different cities in the combination df_train.l3-df_train.l2: 299


In [39]:
l2_l3_cities = []
repeated_cities = []
for x in unique_l2_l3:
    y = x.split(',')
    if y[0] not in l2_l3_cities:
        l2_l3_cities.append(y[0])
    else:
        repeated_cities.append(y[0])
        print(y)


['Granada', ' Meta']
['San Martín', ' Cesar']
['Restrepo', ' Meta']
['Guamal', ' Meta']
['Armenia', ' Antioquia']
['Barbosa', ' Santander']


The output above represents different cities with the same name but in different departments from Colombia. This explains the difference between the amount of unique values in 'l3' and the amount of unique values in the combinatory of 'l2' and 'l3'.

Below you can see the complete list of the aforementioned combinatorics and corroborate that the cities listed above have two entries in the list.

In [40]:
for x in unique_l2_l3:
    y = x.split(',')
    if y[0] in repeated_cities:
        print(x)

Armenia, Quindío
Barbosa, Antioquia
Guamal, Magdalena
Restrepo, Valle del Cauca
Granada, Cundinamarca
San Martín, Meta
Granada, Meta
San Martín, Cesar
Restrepo, Meta
Guamal, Meta
Armenia, Antioquia
Barbosa, Santander


In [41]:
# Now we create a dictionary with the coordinates for each of the unique combinatorics
# This code takes too long to run, so it will be commented out and it's output saved in a dictionary in the helpers.py file.

'''
geolocator = Nominatim(user_agent='acidminded')
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

dep_ciud_lat_lon = {}

for x in capitals.keys():
    dep_ciud_lat_lon[x] = {}

for x in unique_l2_l3:
    coor = geocode(x)
    y = x.split(',')
    ciud = y[0]
    dep = y[1][1:]
    dep_ciud_lat_lon[dep][ciud] = {'lat':coor.latitude, 'lon':coor.longitude}

for x in capitals:
    dep = x
    if capitals[x] not in dep_ciud_lat_lon[x]:
        ciud = capitals[x]
        coor = geocode(f'{ciud}, {dep}')
        dep_ciud_lat_lon[dep][ciud] = {'lat':coor.latitude, 'lon':coor.longitude}

print(dep_ciud_lat_lon)
'''

"\ngeolocator = Nominatim(user_agent='acidminded')\ngeocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)\n\ndep_ciud_lat_lon = {}\n\nfor x in capitals.keys():\n    dep_ciud_lat_lon[x] = {}\n\nfor x in unique_l2_l3:\n    coor = geocode(x)\n    y = x.split(',')\n    ciud = y[0]\n    dep = y[1][1:]\n    dep_ciud_lat_lon[dep][ciud] = {'lat':coor.latitude, 'lon':coor.longitude}\n\nfor x in capitals:\n    dep = x\n    if capitals[x] not in dep_ciud_lat_lon[x]:\n        ciud = capitals[x]\n        coor = geocode(f'{ciud}, {dep}')\n        dep_ciud_lat_lon[dep][ciud] = {'lat':coor.latitude, 'lon':coor.longitude}\n\nprint(dep_ciud_lat_lon)\n"

In [42]:
# Here we check for any problem within our dictionary and found one which is corrected manually.

'''
count = 0
problems = []
for dep in dep_ciud_lat_lon:
    for city in  dep_ciud_lat_lon[dep]:
        if (dep_ciud_lat_lon[dep][city]['lat'] < lat_col[0]) or (dep_ciud_lat_lon[dep][city]['lat'] > lat_col[1]):
            count += 1
            problems.append((dep, city, 'lat problem'))
        if (dep_ciud_lat_lon[dep][city]['lon'] < lon_col[0]) or (dep_ciud_lat_lon[dep][city]['lon'] > lon_col[1]):
            count += 1
            problems.append((dep, city, 'lon problem'))
print(count)
print(problems)
'''

'''
OUTPUT:
1
[('Bolívar', 'Santa Rosa', 'lon problem')]
'''


"\nOUTPUT:\n1\n[('Bolívar', 'Santa Rosa', 'lon problem')]\n"

In [43]:
print(dep_ciud_lat_lon.keys())

dict_keys(['Amazonas', 'Antioquia', 'Arauca', 'Atlántico', 'Bolívar', 'Boyacá', 'Caldas', 'Caquetá', 'Casanare', 'Cauca', 'Cesar', 'Chocó', 'Córdoba', 'Cundinamarca', 'Guainía', 'Guaviare', 'Huila', 'La Guajira', 'Magdalena', 'Meta', 'Nariño', 'Norte de Santander', 'Putumayo', 'Quindío', 'Risaralda', 'San Andrés Providencia y Santa Catalina', 'Santander', 'Sucre', 'Tolima', 'Valle del Cauca', 'Vaupés', 'Vichada'])


In [44]:
# Let's see the missing values from the 'geometry' feature

df_train.geometry.value_counts()

POINT EMPTY                             48519
POINT (-73.112 7.119)                     264
POINT (-75.572 6.203)                     259
POINT (-76.554 3.258)                     199
POINT (-74.1376942 4.6303361)             137
                                        ...  
POINT (-74.8649382591 11.0254292246)        1
POINT (-74.061 4.715)                       1
POINT (-75.43534316 6.02491343)             1
POINT (-74.0868554 4.6703724)               1
POINT (-73.106 7.064)                       1
Name: geometry, Length: 62785, dtype: int64

There are 48519 values missing from 'geometry', but once we have obtained the lat and lon for all our missing columns we can fill in this feature as well.

### 4) PROPERTY FEATURES: rooms, bedrooms, bathrooms, surface_total, surface_covered & property_type

In [45]:
property_features = ['rooms', 'bedrooms', 'bathrooms', 'surface_total', 'surface_covered', 'property_type']

get_info(property_features[:-1], maxmin=True, stats=True)
get_info(property_features[-1:])


----- rooms -----
 •Data types: {<class 'float'>}
 •Missing values:
True     165942
False     27453
Name: rooms, dtype: int64 

 •Min: 1.0
 •Max: 40.0

 •Mean: 3.290387207226897
 •Median: 3.0
 •Mode: 0    3.0
Name: rooms, dtype: float64


----- bedrooms -----
 •Data types: {<class 'float'>}
 •Missing values:
True     153017
False     40378
Name: bedrooms, dtype: int64 

 •Min: 0.0
 •Max: 96.0

 •Mean: 3.241047104859082
 •Median: 3.0
 •Mode: 0    3.0
Name: bedrooms, dtype: float64


----- bathrooms -----
 •Data types: {<class 'float'>}
 •Missing values:
False    153016
True      40379
Name: bathrooms, dtype: int64 

 •Min: 1.0
 •Max: 20.0

 •Mean: 2.638789407643645
 •Median: 2.0
 •Mode: 0    2.0
Name: bathrooms, dtype: float64


----- surface_total -----
 •Data types: {<class 'float'>}
 •Missing values:
True     186453
False      6942
Name: surface_total, dtype: int64 

 •Min: 10.0
 •Max: 180000.0

 •Mean: 1329.7345145491213
 •Median: 120.0
 •Mode: 0    60.0
Name: surface_total, dtype:

From the output above we can see that the only ones of these features that have less than half of it's values missing are 'bathrooms' and 'property_type'.

Because of this, 'bathrooms' and 'property_type' will be the only columns from this subset of features that we will be using for training our models by the moment.

We are awere that it exists the possibility for us to extract meaningful information from each sale's description in order to fill the missing data from these columns and that may be a path we will explore when improving our first models. But for the moment these two features will suffice.

#### The missing values from the 'bathrooms' column will be imputed with it's floor rounded mean value (2), which also happens to be it's median and mode.

In [46]:
df_train.bathrooms.value_counts()

2.0     69152
3.0     30953
1.0     22229
4.0     17019
5.0      7557
6.0      2937
7.0      1158
10.0      928
8.0       681
9.0       329
20.0       14
12.0       14
13.0       11
11.0        8
15.0        7
19.0        6
14.0        5
18.0        5
16.0        2
17.0        1
Name: bathrooms, dtype: int64

In [47]:
df_train.sort_values(by=['bathrooms'], ascending=False).head(15)

,start_date,end_date,created_on,lat,lon,l2,l3,l4,l5,l6,...,bedrooms,bathrooms,surface_total,surface_covered,currency,title,description,property_type,geometry,target
8442,2020-08-05,9999-12-31,2020-08-05,7.114464,-73.119884,Santander,Bucaramanga,NaN,NaN,NaN,...,3.0,20.0,NaN,NaN,COP,"APARTAMENTO EN VENTA, BUCARAMANGA-CONCORDIA",Inmogestion presenta este bonito apartamento e...,Apartamento,POINT (-73.1198842 7.1144635),0
169137,2020-11-27,2020-12-01,2020-11-27,2.811000,-75.227000,Huila,Rivera,NaN,NaN,NaN,...,23.0,20.0,3287.0,500.0,COP,Casa Campestre En Venta En Rivera Cod. VGBX49...,Espectacular Hotel Campestre ubicado en el Cor...,Casa,POINT (-75.227 2.811),1
51712,2020-11-23,2021-08-20,2020-11-23,4.632800,-74.072722,Cundinamarca,Bogotá D.C,Zona Chapinero,Teusaquillo,NaN,...,21.0,20.0,NaN,NaN,COP,"EDIFICIO EN VENTA, BOGOTA-SANTA TERESITA",UBICADÍSIMO EDIFICIO ESQUINERO EN VENTA 21 HA...,Otro,POINT (-74.0727222 4.6328001),1
138694,2020-08-27,9999-12-31,2020-08-27,NaN,NaN,Cundinamarca,Bogotá D.C,Zona Chapinero,Chapinero,Chapinero Central,...,0.0,20.0,NaN,NaN,COP,"EDIFICIO EN VENTA, BOGOTA-CHAPINERO CENTRAL",Edificio para venta en el sector de Chapinero....,Otro,POINT EMPTY,1
185592,2020-09-12,2020-11-13,2020-09-12,3.421000,-76.545000,Valle del Cauca,Cali,NaN,NaN,NaN,...,20.0,20.0,210.0,692.0,COP,Edificio En Arriendo/venta En Cali Urbanizacin...,"Se Vende o Se alquila, edificio en la zona sur...",Otro,POINT (-76.545 3.421),1
53449,2020-11-30,2021-07-01,2020-11-30,4.644464,-74.060546,Cundinamarca,Bogotá D.C,Zona Chapinero,Chapinero,NaN,...,22.0,20.0,NaN,NaN,COP,"CASA EN VENTA, BOGOTA-CHAPINERO",Casa esquinera remodelada. ambiente acogedor. ...,Casa,POINT (-74.060546 4.6444642),1
39286,2020-09-02,9999-12-31,2020-09-02,4.809850,-74.101838,Cundinamarca,Cota,NaN,NaN,NaN,...,0.0,20.0,NaN,NaN,COP,"BODEGA EN VENTA, COTA-COTA","Bodegas 545.04 Mts2 y oficinas 245,28. Mts2 pa...",Otro,POINT (-74.1018383 4.8098502),1
96152,2020-09-30,2021-01-18,2020-09-30,4.680884,-74.130453,Cundinamarca,Bogotá D.C,Zona Occidental,Fontibón,NaN,...,0.0,20.0,NaN,NaN,COP,"EDIFICIO EN VENTA, BOGOTA-EL DORADO",Excelente ubicación muy cerca de estaciones de...,Otro,POINT (-74.1304534018 4.6808835909),1
69334,2020-09-10,2021-05-24,2020-09-10,4.637380,-74.064302,Cundinamarca,Bogotá D.C,Zona Chapinero,Chapinero,Marly,...,45.0,20.0,NaN,NaN,COP,"EDIFICIO EN ARRIENDO/VENTA, BOGOTA-MARLY",espectacular edificio de 4000 mtrs2 construido...,Otro,POINT (-74.0643016 4.6373802),0
42219,2020-08-28,2020-12-02,2020-08-28,NaN,NaN,Cundinamarca,Bogotá D.C,Zona Centro,La Candelaria,Las Aguas,...,96.0,20.0,NaN,NaN,COP,"HOTEL EN VENTA, BOGOTA-LAS AGUAS",Perfecta ubicación en el centro de Bogotá. A t...,Otro,POINT EMPTY,1


In [48]:
df_train.property_type.value_counts()

Apartamento        98687
Casa               59114
Otro               16152
Lote               15981
Local comercial     1248
Finca               1131
Oficina             1072
Parqueadero           10
Name: property_type, dtype: int64

In [169]:
property_types = df_train.property_type.unique()
print(property_types)

['Casa' 'Apartamento' 'Otro' 'Oficina' 'Finca' 'Lote' 'Local comercial'
 'Parqueadero']


In [53]:
print(f'\nHouse or apartment registries by amount of bathrooms:\n')
for x in range(5,21):
    print(f'• {x} bathrooms:')
    print('\t',len(df_train.loc[((df_train.property_type == 'Casa') | (df_train.property_type == 'Apartamento'))&((df_train.bathrooms >= x))]))


Houses and apartments by amount of bathrooms:

• 5 bathrooms:
	 11061
• 6 bathrooms:
	 4390
• 7 bathrooms:
	 2045
• 8 bathrooms:
	 1134
• 9 bathrooms:
	 661
• 10 bathrooms:
	 412
• 11 bathrooms:
	 30
• 12 bathrooms:
	 26
• 13 bathrooms:
	 20
• 14 bathrooms:
	 13
• 15 bathrooms:
	 10
• 16 bathrooms:
	 7
• 17 bathrooms:
	 7
• 18 bathrooms:
	 7
• 19 bathrooms:
	 6
• 20 bathrooms:
	 4


It is very unlikely that a house or an apartment will have 6 or more bathrooms, for this reason, those values will be replaced by the floor rounded mean of the column (2).

### 5) ADVERTISING FEATURES: currency, title & description 

In [54]:
advertising_features = ['currency', 'title', 'description']

get_info(advertising_features)


----- currency -----
 •Data types: {<class 'str'>, <class 'float'>}
 •Missing values:
False    193391
True          4
Name: currency, dtype: int64 


----- title -----
 •Data types: {<class 'str'>, <class 'float'>}
 •Missing values:
False    193394
True          1
Name: title, dtype: int64 


----- description -----
 •Data types: {<class 'str'>, <class 'float'>}
 •Missing values:
False    193274
True        121
Name: description, dtype: int64 



In [55]:
df_train[['currency', 'title', 'description']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193395 entries, 0 to 197548
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   currency     193391 non-null  object
 1   title        193394 non-null  object
 2   description  193274 non-null  object
dtypes: object(3)
memory usage: 5.9+ MB


In [56]:
df_train[df_train['currency'].isnull()]

,start_date,end_date,created_on,lat,lon,l2,l3,l4,l5,l6,...,bedrooms,bathrooms,surface_total,surface_covered,currency,title,description,property_type,geometry,target
16240,2020-11-27,2020-11-27,2020-11-27,NaN,NaN,Valle del Cauca,Cali,NaN,NaN,NaN,...,6.0,2.0,NaN,NaN,NaN,Venta casa barrio cuidad Córdoba cali vall,<b>Venta casa barrio cuidad Córdoba cali vall...,Casa,POINT EMPTY,0
53528,2020-12-15,2020-12-15,2020-12-15,NaN,NaN,Tolima,Ibagué,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,VENDO APARTAMENTO BALCONES DEL VERGEL,<b>VENDO APARTAMENTO BALCONES DEL VERGEL</b><...,Apartamento,POINT EMPTY,0
90818,2020-10-21,2020-10-21,2020-10-21,NaN,NaN,Antioquia,Sabaneta,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,Cod12 Aparatamento en Venta Poblado,<br />\n <br />\n Ref#582518.,Apartamento,POINT EMPTY,0
119682,2020-10-21,2020-10-21,2020-10-21,NaN,NaN,Santander,Bucaramanga,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Vendo Casa Campestre,<br />\n <br />\n Ref#582835.,Finca,POINT EMPTY,0


In [57]:
# Here we can see that 8 of the registers have a price in usd
df_train.currency.value_counts()

COP    193383
USD         8
Name: currency, dtype: int64

In [58]:
df_train.loc[df_train.currency=='USD']

,start_date,end_date,created_on,lat,lon,l2,l3,l4,l5,l6,...,bedrooms,bathrooms,surface_total,surface_covered,currency,title,description,property_type,geometry,target
5902,2020-08-25,9999-12-31,2020-08-25,4.914310,-73.993189,Cundinamarca,Sopó,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,USD,Vendo espectacular casa entre Bogota y Briceño,Espectacular casa entre Bogota- Briceño km 12 ...,Casa,POINT (-73.9931885 4.9143096),0
49196,2020-10-07,2021-01-22,2020-10-07,10.513831,-75.498685,Bolívar,Cartagena,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,USD,Exclusive beach house for sale - Manzanillo de...,¡EXCLUSIVE BEACH HOUSE FOR SALE - MANZANILLO D...,Casa,POINT (-75.4986852407 10.5138313669),0
56159,2020-10-26,2020-10-26,2020-10-26,4.739003,-74.098302,Cundinamarca,Bogotá D.C,Zona Noroccidental,NaN,NaN,...,5.0,NaN,NaN,NaN,USD,Casa en Venta Costa del Este RAH PA: 20-11172,Viva en una casa espaciosa con terrazas y pati...,Casa,POINT (-74.098302 4.7390028),0
56522,2020-10-26,2020-10-26,2020-10-26,4.695757,-74.043894,Cundinamarca,Bogotá D.C,Zona Norte,Usaquén,NaN,...,4.0,NaN,NaN,NaN,USD,Apartamento en Venta Santa Maria RAH PA: 20-10683,Majestuoso apartamento a estrenar con la mejor...,Apartamento,POINT (-74.0438943 4.6957568),0
116979,2020-09-13,2021-01-12,2020-09-13,NaN,NaN,Santander,Bucaramanga,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,USD,Villa for sale Bali,Villa for sale Bali<br />\n<br />\nLocation: J...,Finca,POINT EMPTY,0
136236,2020-09-16,2020-10-29,2020-09-16,4.622794,-74.090960,Cundinamarca,Bogotá D.C,Zona Centro,Puente Aranda,NaN,...,3.0,2.0,NaN,NaN,USD,Venta Casa Excelente,<br />\n - Calefacción\n- Parrilla\n <br />\n ...,Casa,POINT (-74.0909602 4.622794),0
137025,2020-08-29,9999-12-31,2020-08-29,12.585979,-81.714549,San Andrés Providencia y Santa Catalina,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,USD,HOTEL EN VENTA EN LA ISLA DE SAN ANDRÉS,\nUn Hotel Boutique TOTALMENTE frente al mar.\...,Otro,POINT (-81.7145490646 12.5859785199),0
167143,2020-08-08,9999-12-31,2020-08-08,10.829302,-75.160260,Atlántico,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,USD,Lote en venta Vía Barranquilla Cartagena,OPORTUNIDAD DE INVERSIÒN EN EL CARIBE COLOMBIA...,Lote,POINT (-75.1602602005 10.8293016581),0


In [71]:
df_train.description.duplicated().value_counts()


False    111313
True      82082
Name: description, dtype: int64

In [60]:
df_train.columns

Index(['start_date', 'end_date', 'created_on', 'lat', 'lon', 'l2', 'l3', 'l4',
       'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms', 'surface_total',
       'surface_covered', 'currency', 'title', 'description', 'property_type',
       'geometry', 'target'],
      dtype='object')

In [72]:
df_train_nd = df_train[['lat', 'lon', 'l2', 'l3', 'l4',
       'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms', 'surface_total',
       'surface_covered', 'currency', 'title', 'description', 'property_type',
       'geometry', 'target']].copy()
df_train_nd.duplicated().value_counts()

False    120377
True      73018
dtype: int64

In [73]:
df_train_nd.description.duplicated().value_counts()

False    111313
True      82082
Name: description, dtype: int64

In [74]:
df_train_nd.drop_duplicates(inplace=True)
df_train_nd.description.duplicated().value_counts()

False    111313
True       9064
Name: description, dtype: int64

In [70]:
df_train_nd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120377 entries, 0 to 197545
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   lat              105851 non-null  float64
 1   lon              105851 non-null  float64
 2   l2               120377 non-null  object 
 3   l3               110748 non-null  object 
 4   l4               34480 non-null   object 
 5   l5               20377 non-null   object 
 6   l6               5910 non-null    object 
 7   rooms            26914 non-null   float64
 8   bedrooms         39718 non-null   float64
 9   bathrooms        87712 non-null   float64
 10  surface_total    6922 non-null    float64
 11  surface_covered  9723 non-null    float64
 12  currency         120373 non-null  object 
 13  title            120376 non-null  object 
 14  description      120257 non-null  object 
 15  property_type    120377 non-null  object 
 16  geometry         120377 non-null  obje

In [75]:
dft = df_train.copy()
dft.shape

(193395, 21)

In [76]:
dft[['lat', 'lon', 'l2', 'l3', 'l4',
       'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms', 'surface_total',
       'surface_covered', 'currency', 'title', 'description', 'property_type',
       'geometry', 'target']].duplicated().value_counts()

False    120377
True      73018
dtype: int64

In [78]:
dft.drop_duplicates(subset=['lat', 'lon', 'l2', 'l3', 'l4',
       'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms', 'surface_total',
       'surface_covered', 'currency', 'title', 'description', 'property_type',
       'geometry', 'target'], inplace=True)
dft.shape

(120377, 21)

In [79]:
dft.columns

Index(['start_date', 'end_date', 'created_on', 'lat', 'lon', 'l2', 'l3', 'l4',
       'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms', 'surface_total',
       'surface_covered', 'currency', 'title', 'description', 'property_type',
       'geometry', 'target'],
      dtype='object')

In [80]:
dft.duplicated().value_counts()

False    120377
dtype: int64

In [201]:
#dft.surface_total.isnull().value_counts()
#dft.info()

## -------------- D A T A --- P R E P R O C E S S I N G --- 2 --------------

### ------------------------------------ CREATING THE PIPELINE ------------------------------------

We will design a pipeline the recieves a dataset with the same features as the one we just explored ('df_train') as it was at the beginning of the previous section (minus the target column). This pipeline will perform the necessary changes to the dataset, feed it to a model of our selection, perform a cross validation and give us the results.

As we concluded on the previous section, we will select a few features that will be considered relevant to continue with the data preprocessing and model training: l2, l3, lat, lon, bathrooms and property_type.

l2--- categorical (needs encoding). MissVal (0, ok!)

l3--- categorical (needs encoding). MissVal (needs imputation using 'capitals')

lat--- numerical (ok). MissVal (needs imputation using 'dep_ciud_lat_lon') -standard scaler

lon--- numerical (ok). MissVal (needs imputation using 'dep_ciud_lat_lon')

bathrooms-- numerical (ok). MissVal (needs imputation using mean 2). Replace values greater than 5 (and with property type 'casa' or 'apartamento) with 2 by 2

property_type--- categorical (needs encoding). MissVal (0, ok!)

In [404]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score
from sklearn.model_selection import train_test_split

In [255]:
X1 = dft.drop('target',axis=1)
y1 = dft.target

#### Here we define some helper functions that will be used to fill missing values during the preprocessing

In [257]:
def fill_l3(df):
    l3_ok = []
    for x in range(len(df)):
        if type(df.loc[x,'l3']) == float:
            dep = df.loc[x,'l2']
            l3_ok.append(str(capitals[dep]))
        else:
            l3_ok.append(str(df.loc[x,'l3']))
    return pd.Series(l3_ok)
        

In [258]:
def fill_coor(df):
    lat_ok = []
    lon_ok = []
    for x in range(len(df)):
        if str(df.loc[x,'lat']) == 'nan':
            dep, city = df.loc[x,'l2':'l3']
            #print('NAN FOUND: ',dep, city, df.loc[x,'lat'], df.loc[x,'lon'])
            lat_ok.append(float(dep_ciud_lat_lon[dep][city]['lat']))
            lon_ok.append(float(dep_ciud_lat_lon[dep][city]['lon']))
        else:
            lat_ok.append(df.loc[x,'lat'])
            lon_ok.append(df.loc[x,'lon'])
    return pd.Series(lat_ok), pd.Series(lon_ok)

In [259]:
prtypes = ['Casa', 'Apartamento', 'Otro', 'Oficina', 'Finca', 'Lote', 'Local comercial', 'Parqueadero']
prtypes_avgbtrms = {}

for x in prtypes:
    prtypes_avgbtrms[x] = X1.loc[X1.property_type==x].bathrooms.mean()
    if str(prtypes_avgbtrms[x]) == 'nan':
        prtypes_avgbtrms[x] = 1
for x in prtypes_avgbtrms:
    prtypes_avgbtrms[x] = round(prtypes_avgbtrms[x])

def fill_bathrooms(df):  #hehe
    btrms_ok = []
    for x in range(len(df)):
        btrms = df.loc[x,'bathrooms'] 
        prtype = df.loc[x,'property_type']

        if str(btrms) == 'nan':
            btrms_ok.append(prtypes_avgbtrms[prtype])
        elif btrms >= 6:
            if prtype in ['Casa', 'Apartamento']:
                btrms_ok.append(float(2))
            else:
                btrms_ok.append(btrms)
        else:
            btrms_ok.append(btrms)
    return pd.Series(btrms_ok)

In [261]:
#std_scaler = StandardScaler().fit(X1[['lat','lon']].to_numpy()[:,:])
#min_max_scaler = MinMaxScaler().fit(X1[['bathrooms']].to_numpy()[:,:])
print(X1.to_numpy().shape)

(120377, 20)


In [375]:
def fill_nan(dataset):
    X = dataset[['bathrooms', 'lat', 'lon', 'l2', 'l3', 'property_type']].copy()
    X.reset_index(inplace=True, drop=True)
    # Fill missing values
    X['l3'] = fill_l3(X)
    X['lat'], X['lon'] = fill_coor(X)
    X['bathrooms'] = fill_bathrooms(X)
    # Prepare to transform to numerical
    X['l2'] = X['l2'].astype('category')
    X['l3'] = X['l3'].astype('category')
    X['property_type'] = X['property_type'].astype('category')
    return X

In [376]:
X2 = fill_nan(X1)
X2.isnull().sum()

bathrooms        0
lat              0
lon              0
l2               0
l3               0
property_type    0
dtype: int64

In [393]:
l2_coder = OneHotEncoder()
l3_coder = OneHotEncoder()
pt_coder = OneHotEncoder()
l2_cod = l2_coder.fit(X2[['l2']])
l3_cod = l3_coder.fit(X2[['l3']])
pt_cod = pt_coder.fit(X2[['property_type']])


def preprocess_to_num(dataset):
    #print(f'INITIAL TO_NUM SHAPE: {dataset.shape}')
    
    X = dataset.copy()
    
    #print(f'TO_NUM SHAPE BEFORE ONEHOT: {X.shape}')

    #l2_coder = OneHotEncoder()
    #l2_cod = l2_coder.fit_transform(X[['l2']])
    l2_cod = l2_coder.transform(X[['l2']])
    new_l2 = pd.DataFrame(l2_cod.toarray(), columns=l2_coder.categories_)
    #print(f'+{new_l2.shape} (l2)')

    #l3_coder = OneHotEncoder()
    #l3_cod = l3_coder.fit_transform(X[['l3']])
    l3_cod = l3_coder.transform(X[['l3']])
    new_l3 = pd.DataFrame(l3_cod.toarray(), columns=l3_coder.categories_)
    #print(f'+{new_l3.shape} (l3)')

    #pt_coder = OneHotEncoder()
    #pt_cod = pt_coder.fit_transform(X[['property_type']])
    pt_cod = pt_coder.transform(X[['property_type']])
    new_pt = pd.DataFrame(pt_cod.toarray(), columns=pt_coder.categories_)
    #print(f'+{new_pt.shape} (pt)')

    #print(f'TO_NUM SHAPE AFTER ONEHOT: {X.shape}')

    X_ok = pd.concat([X, new_l2, new_l3, new_pt], axis=1)
    X_ok.drop(['l2','l3','property_type'], axis=1, inplace=True)

    X_num = X_ok.to_numpy()
    #print(f'FINAL TO_NUM SHAPE: {X_num.shape}')

    return X_num
    

X_num = preprocess_to_num(X2)

In [379]:
print(X_num.shape)
X_num

(120377, 339)


array([[  4.       ,   6.203    , -75.572    , ...,   0.       ,
          0.       ,   0.       ],
       [  2.       ,   4.7227485, -74.0731146, ...,   0.       ,
          0.       ,   0.       ],
       [  2.       ,   4.709    , -74.03     , ...,   0.       ,
          0.       ,   0.       ],
       ...,
       [  2.       ,   7.064    , -73.106    , ...,   0.       ,
          0.       ,   0.       ],
       [  2.       ,   4.649682 , -74.1619504, ...,   0.       ,
          0.       ,   0.       ],
       [  1.       ,   4.668    , -74.055    , ...,   1.       ,
          0.       ,   0.       ]])

In [394]:
min_max_scaler = MinMaxScaler().fit(X_num[:,:1])
std_scaler = StandardScaler().fit(X_num[:,1:])
pca = PCA(n_components=30, whiten=False).fit(X_num)

def preprocess_std_dimred(matrix):
    #print(f' STANDARDIZATION INITIAL SHAPE: {matrix.shape}')
    X =  np.copy(matrix)
    X[:,:1] = min_max_scaler.transform(matrix[:,:1])
    X[:,1:] = std_scaler.transform(matrix[:,1:])
    #print(f' STANDARDIZATION INTERMEDIATE SHAPE: {X.shape}')
    X = pca.transform(X)
    #print(f' STANDARDIZATION FINAL SHAPE: {X.shape}')
    return X
    

In [395]:
X_ready = preprocess_std_dimred(X_num)
X_ready.shape

(120377, 30)

In [396]:
fill_df = FunctionTransformer(fill_nan)
df_to_num = FunctionTransformer(preprocess_to_num)
mat_to_X = FunctionTransformer(preprocess_std_dimred)

In [409]:
clf_kn = KNeighborsClassifier(n_neighbors=10)

In [414]:
clf_dt = DecisionTreeClassifier(max_depth=8)

In [410]:
pipeline_1 = Pipeline([('Fill_DF', fill_df),('To_Num', df_to_num),('Standardize_&_DimRed', mat_to_X), ('KNClassifier', clf_kn)])
pipeline_1

Pipeline(steps=[('Fill_DF',
                 FunctionTransformer(func=<function fill_nan at 0x0000020BC11ACAF0>)),
                ('To_Num',
                 FunctionTransformer(func=<function preprocess_to_num at 0x0000020C1A6B0AF0>)),
                ('Standardize_&_DimRed',
                 FunctionTransformer(func=<function preprocess_std_dimred at 0x0000020C1A6B0670>)),
                ('KNClassifier', KNeighborsClassifier(n_neighbors=10))])

In [415]:
pipeline_2 = Pipeline([('Fill_DF', fill_df),('To_Num', df_to_num),('Standardize_&_DimRed', mat_to_X), ('DTClassifier', clf_dt)])

In [411]:
def fit_and_print(pipeline, X_train, y_train, X_test, y_test):
    #print(f'SHAPE TO FIT: {X_train.shape}')
    pipeline.fit(X_train, y_train)
    train_preds = pipeline.predict(X_train)
    test_preds = pipeline.predict(X_test)
    print('• TRAIN DATA:')
    print(f'Train confusion matrix: \n{confusion_matrix(y_train, train_preds)}')
    print(f'Train accuracy: {accuracy_score(y_train, train_preds)}')
    print(f'Train recall: {recall_score(y_train, train_preds)}\n\n')

    print('• TEST DATA:')
    print(f'Test confusion matrix: \n{confusion_matrix(y_test, test_preds)}')
    print(f'Test accuracy: {accuracy_score(y_test, test_preds)}')
    print(f'Test recall: {recall_score(y_test, test_preds)}')

In [412]:
X_train, X_test, y_train, y_test = train_test_split(X2, y1, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((96301, 6), (96301,), (24076, 6), (24076,))

In [413]:
# Kneighbors classifier n_neighbors=10
fit_and_print(pipeline_1, X_train, y_train, X_test, y_test)

• TRAIN DATA:
Train confusion matrix: 
[[68396  3762]
 [ 8977 15166]]
Train accuracy: 0.8677168461386694
Train recall: 0.6281737977881787


• TEST DATA:
Test confusion matrix: 
[[16839  1204]
 [ 2503  3530]]
Test accuracy: 0.8460292407376641
Test recall: 0.585115199734792


In [416]:
# Decision tree classifier max_depth=8
fit_and_print(pipeline_2, X_train, y_train, X_test, y_test)

• TRAIN DATA:
Train confusion matrix: 
[[67744  4414]
 [12046 12097]]
Train accuracy: 0.8290775796720699
Train recall: 0.5010562067680073


• TEST DATA:
Test confusion matrix: 
[[16807  1236]
 [ 3001  3032]]
Test accuracy: 0.8240156172121615
Test recall: 0.5025692027183822
